# Word2Vec then RNN

## Load data

In [1]:
import pandas as pd
import numpy as np
from os import path

importDirectory = "../state/data/preprocessed-train-test/"

train, test, data, contestTest = map(
    lambda filename: pd.read_csv(path.join(importDirectory, filename)), 
    ["train.csv", "test.csv", "all.csv", "contest-test.csv"])

In [2]:
print("train: {}, test: {}, all: {}, contestTest: {}".format(
    train.shape, test.shape, data.shape, contestTest.shape))

train: (127656, 8), test: (31915, 8), all: (159571, 8), contestTest: (153164, 2)


In [3]:
def splitSentences(dataset):
    return (dataset.comment_text
    .str.replace("[^A-Za-z\s]", "")
    .str.lower()
    .str.split())

In [4]:
%%time
splitTrain = splitSentences(train)
splitTest = splitSentences(test)

CPU times: user 2.92 s, sys: 168 ms, total: 3.09 s
Wall time: 3.1 s


## Convert words to integers

In [5]:
from sklearn.preprocessing import LabelEncoder

allWords = set([word for sentence in pd.concat([splitTrain, splitTest]) for word in sentence])

In [6]:
wordToInteger = { word: index for index, word in enumerate(allWords) }
integerToWord = { index: word for index, word in enumerate(allWords) }

In [7]:
integerTrain = splitTrain.apply(lambda sentence: [wordToInteger[word] for word in sentence])
integerTest = splitTest.apply(lambda sentence: [wordToInteger[word] for word in sentence])

## Int2Vec embedding matrix

In [8]:
numDimensions = 50
maxSeqLength = 250

In [9]:
%%time
import gensim

w2vModel = gensim.models.KeyedVectors.load_word2vec_format("../state/external-models/glove.6B/w2v.glove.6B.50.txt")

CPU times: user 22.6 s, sys: 360 ms, total: 22.9 s
Wall time: 23.1 s


In [10]:
availableWords = set.intersection(allWords, set(w2vModel.vocab.keys()))

In [11]:
np.random.seed(432432)

int2vec = {index: w2vModel.word_vec(word) 
             if word in availableWords 
             else np.random.normal(scale=.644, size=(numDimensions,))
         for index, word in integerToWord.items()}

In [12]:
embeddingMatrix = np.array([vector for vector in int2vec.values()])

## Oversample

In [13]:
def oversample(dataset, label): 
    multiples = int(dataset[dataset[label] == 0].shape[0] / dataset[dataset[label] == 1].shape[0])
    
    datasetPositive = dataset[dataset[label] == 1]
    
    return pd.concat([dataset] + multiples * [datasetPositive]).reset_index()

In [14]:
trainOversampled = oversample(train, "toxic")

## Zero pad vectors

In [16]:
def padArrayWithZeros(array):
    fullArray = np.zeros(maxSeqLength)
    fullArray[:min(array.shape[0], maxSeqLength)] = array[:min(array.shape[0], maxSeqLength)]
    return fullArray

In [17]:
def prepareText(dataset):
    return np.array(splitSentences(dataset)
        .apply(lambda sentence: 
            padArrayWithZeros(np.array([
                wordToInteger[word]
                for word in sentence])))
        .tolist())

In [18]:
%%time
trainSentences = prepareText(trainOversampled)

CPU times: user 10.3 s, sys: 492 ms, total: 10.8 s
Wall time: 10.8 s


In [19]:
trainLabels = np.array(trainOversampled
    .toxic
    .apply(lambda label: np.array([0, 1]) if label == 1 else np.array([1, 0]))
    .tolist())

In [20]:
testSentences = prepareText(test)

In [21]:
testLabels = np.array(test
    .toxic
    .apply(lambda label: np.array([0, 1]) if label == 1 else np.array([1, 0]))
    .tolist())

In [ ]:
# Small datasets
smallSize = 10000
trainSentences = trainSentences[:smallSize]
trainLabels = trainLabels[:smallSize]
testSentences = trainSentences[:smallSize]
testLabels = trainLabels[:smallSize]

## LMST RNN with keras

In [22]:
lstmUnits = [100]
numClasses = 2
batchSize = 1024

In [23]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15567104157979966545
]


In [24]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, TimeDistributed, Dropout, Embedding
from keras.optimizers import Adam

Using TensorFlow backend.


In [25]:
import keras.backend as K
import tensorflow as tf

def auc(y_true, y_pred):
     auc = tf.metrics.auc(y_true, y_pred)[1]
     K.get_session().run(tf.local_variables_initializer())
     return auc

In [28]:
model = Sequential()

model.add(Embedding(
    embeddingMatrix.shape[0],
    embeddingMatrix.shape[1],
    weights=[embeddingMatrix],
    input_length=maxSeqLength,
    trainable=False))

model.add(LSTM(lstmUnits[0], name="LSTM"))

model.add(Dropout(.2, name="dropout"))

model.add(Dense(
    2, 
    activation="softmax", 
    name="softmax_output"))

model.compile(
    loss="categorical_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy", auc])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 50)           11198300  
_________________________________________________________________
LSTM (LSTM)                  (None, 100)               60400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
softmax_output (Dense)       (None, 2)                 202       
Total params: 11,258,902
Trainable params: 60,602
Non-trainable params: 11,198,300
_________________________________________________________________
None


In [33]:
tf.set_random_seed(43245)
np.random.seed(453252)

In [29]:
%%time
model.fit(
    trainSentences, 
    trainLabels, 
    nb_epoch=2, 
    batch_size=batchSize,
    validation_data=(testSentences, testLabels))

/home/mariosk/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 10000 samples, validate on 10000 samples
Epoch 1/2
10000/10000 [==============================] - 46s 5ms/step - loss: 0.4733 - acc: 0.7681 - auc: 0.6317 - val_loss: 0.3694 - val_acc: 0.9017 - val_auc: 0.8784
Epoch 2/2
10000/10000 [==============================] - 45s 5ms/step - loss: 0.3491 - acc: 0.9022 - auc: 0.8920 - val_loss: 0.3217 - val_acc: 0.9024 - val_auc: 0.8964
CPU times: user 5min 18s, sys: 17.4 s, total: 5min 36s
Wall time: 1min 32s


In [ ]:
preds = model.predict(testSentences, batch_size=batchSize)
prdVec = np.where(preds[:, 1] > 0.5, 1, 0)
labs = testLabels[:, 1]

In [ ]:
from evaluate_predictions import evaluatePredictions

evaluatePredictions(pd.Series(labs), prdVec, preds)